Trước khi run pip, cài đặt các thư viện bằng câu lệnh sau
```
pip install dash
```

In [1]:
# Cài đặt các thư viện cần thiết
import dash
from dash import dcc
from dash import  html
import pandas as pd
import plotly.graph_objs as go
from dash.dependencies import Input, Output
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))

import numpy as np

# Khởi tạo ứng dụng Dash
app = dash.Dash()
server = app.server

# Đọc dữ liệu, sắp xếp theo ngày, lấy cột ngày và giá đóng cửa
df_nse = pd.read_csv("NSE-TATA.csv")

df_nse["Date"]=pd.to_datetime(df_nse.Date,format="%Y-%m-%d")
df_nse.index=df_nse['Date'].values

data=df_nse.sort_index(ascending=True,axis=0)
new_data=pd.DataFrame(index=range(0,len(df_nse)),columns=['Date','Close'])

for i in range(0,len(data)):
    new_data.loc[i, "Date"]=data['Date'].iloc[i]
    new_data.loc[i, "Close"]=data["Close"].iloc[i]

new_data.index=new_data['Date'].values
new_data.drop("Date",axis=1,inplace=True)
new_data.head()

# Chuẩn hóa dữ liệu
dataset=new_data.values

train=dataset[0:987,:]
valid=dataset[987:,:]

scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(dataset)

x_train,y_train=[],[]

for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])

x_train,y_train=np.array(x_train),np.array(y_train)

x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))

# Lấy mẫu từ dữ liệu để đưa ra dự đoán giá cổ phiếu bằng mô hình LSTM:

model=load_model("saved_lstm_model.keras")

inputs=new_data[len(new_data)-len(valid)-60:].values
inputs=inputs.reshape(-1,1)
inputs=scaler.transform(inputs)

X_test=[]
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test=np.array(X_test)

X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
closing_price=model.predict(X_test)
closing_price=scaler.inverse_transform(closing_price)

train=new_data[:987]
valid=new_data[987:]

valid.loc[:, 'Predictions_LSTM'] = closing_price

print(valid)

# Lấy mẫu từ dữ liệu để đưa ra dự đoán giá cổ phiếu bằng mô hình RNN:

model=load_model("saved_rnn_model.keras")

inputs=new_data[len(new_data)-len(valid)-60:].values
inputs=inputs.reshape(-1,1)
inputs=scaler.transform(inputs)

X_test=[]
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test=np.array(X_test)

X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

closing_price=model.predict(X_test)
closing_price=scaler.inverse_transform(closing_price)

valid.loc[:, 'Predictions_RNN'] = closing_price

print(valid)

NameError: name 'valid' is not defined

In [ ]:
df= pd.read_csv("stock_data.csv")

app.layout = html.Div([

    html.H1("Stock Price Analysis Dashboard", style={"textAlign": "center"}),

    dcc.Tabs(id="tabs", children=[

        dcc.Tab(label='NSE-TATAGLOBAL Actual closing price',children=[
			html.Div([
				html.H2("Actual closing price",style={"textAlign": "center"}),
				dcc.Graph(
					id="Actual Data",
					figure={
						"data":[
							go.Scatter(
								x=valid.index,
								y=valid["Close"],
								mode='markers'
							)

						],
						"layout":go.Layout(
							title='scatter plot',
							xaxis={'title':'Date'},
							yaxis={'title':'Closing Rate'}
						)
					}

				),
			])


        ]),
        dcc.Tab(label='NSE-TATAGLOBAL prediction closing price', children=[
            html.Div([
                html.H1("Prediction closing price",
                        style={'textAlign': 'center'}),

                dcc.Dropdown(id='my-dropdown',
                             options=[{'label': 'LSTM', 'value': 'LSTM'},
                                      {'label': 'RNN','value': 'RNN'}],
                             multi=True,value=['LSTM'],
                             style={"display": "block", "margin-left": "auto",
                                    "margin-right": "auto", "width": "60%"}),
                dcc.Graph(id='prediction'),

            ]),
        ])


    ])
])

@app.callback(Output('prediction', 'figure'),
              [Input('my-dropdown', 'value')])
def update_graph(selected_dropdown):
    dropdown = {"LSTM": "LSTM","RNN": "RNN"}
    trace1 = []
    trace2 = []
    
    for model in selected_dropdown:
        trace1.append(
          go.Scatter(x=valid.index,
					 y=valid["Predictions" + "_" + model],
                     mode='lines', opacity=0.7,
                     name=f'Predictions_LSTM',textposition='bottom center'))
        

    traces = [trace1, trace2]
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data,
              'layout': go.Layout(colorway=["#5E0DAC", '#FF4F00', '#375CB1',
                                            '#FF7400', '#FFF400', '#FF0056'],
            height=600,
            title=f"High and Low Prices for {', '.join(str(dropdown[i]) for i in selected_dropdown)} Over Time",
            xaxis={"title":"Date",
                   'rangeselector': {'buttons': list([{'count': 1, 'label': '1M',
                                                       'step': 'month',
                                                       'stepmode': 'backward'},
                                                      {'count': 6, 'label': '6M',
                                                       'step': 'month',
                                                       'stepmode': 'backward'},
                                                      {'step': 'all'}])},
                   'rangeslider': {'visible': True}, 'type': 'date'},
             yaxis={"title":"Price (USD)"})}
    return figure


if __name__=='__main__':
	app.run_server(debug=True)